In [1]:
import json 
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
from elasticsearch import Elasticsearch
import time

In [2]:
access_token= 'your token'
access_secret='your token'
consumer_key='your token'
consumer_secret='your token'
#APIkey=''
#APisecretkey=''
#Bearer token=''

In [3]:
#import twitter keys and tokens 
#from config import *

#new instance for ElasticSearch
es= Elasticsearch()

class TweetStreamListener(StreamListener):
    def __init__(self, time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        super(TweetStreamListener, self).__init__()
    
    #when we are able to listen to data
    def on_data(self,data):
        if (time.time() - self.start_time) < self.limit:
            dict_data=json.loads(data)
            tweet= TextBlob(dict_data["text"])#text is a field in twitter data being listened.

            print(tweet.sentiment.polarity)

            if(tweet.sentiment.polarity<0):
                sentiment="negative"
            elif(tweet.sentiment.polarity==0):
                sentiment="neutral"
            else:
                sentiment="positive"

            print(sentiment)

            es.index(index="sentiment",
                    doc_type="test-type",
                    body={"author": dict_data["user"]["screen_name"],
                         "date": dict_data["created_at"],
                         "message": dict_data["text"],
                         "polarity": tweet.sentiment.polarity,
                         "subjectivity": tweet.sentiment.subjectivity,
                         "sentiment": sentiment})
            return True

        else:
            return False
    
    
    def on_error(self,status):
        print(status)

if __name__ == "__main__":

    listener = TweetStreamListener(time_limit=30)
    auth= OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    runtime =60
    stream = Stream(auth,listener)

    stream.filter(track=['biden'])
    


    

0.7
positive
-0.6999999999999998
negative


C:\Users\kapis\anaconda3\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


0.25
positive
-0.2
negative
0.0
neutral
0.25
positive
0.3666666666666667
positive
0.0
neutral
-0.5
negative
0.0
neutral
0.3
positive
0.0
neutral
0.0
neutral
0.3
positive
0.0
neutral
0.3
positive
0.0
neutral
0.0
neutral
0.575
positive
-0.44999999999999996
negative
-0.3
negative
0.0
neutral
0.0
neutral
0.0
neutral
-0.08125
negative
0.8
positive
0.1
positive
0.0
neutral
0.06666666666666667
positive
0.8
positive
0.0
neutral
0.0
neutral
0.0
neutral
0.3666666666666667
positive
-0.4
negative
0.5
positive
0.0
neutral
0.0
neutral
0.0
neutral
0.0
neutral
-0.016666666666666663
negative
0.0
neutral
0.35
positive
0.0
neutral
0.0
neutral
-0.16666666666666666
negative
0.0
neutral
0.5
positive
0.0
neutral
-0.3333333333333333
negative
0.0
neutral
0.0
neutral
0.0
neutral
0.3
positive
-0.33888888888888885
negative
0.0
neutral
0.0
neutral
0.0
neutral
-0.022222222222222227
negative
0.0
neutral
0.3111111111111111
positive
0.0
neutral
0.06818181818181818
positive
0.1
positive
0.0
neutral
0.0
neutral
0.136363

In [ ]:
es = Elasticsearch(['http://localhost:9200/'], verify_certs=True)

if not es.ping():
    raise ValueError("Connection failed")